In [1]:
spark = SparkSession \
        .builder \
        .appName("HWE Week 1 App") \
        .getOrCreate()

23/06/11 17:09:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.sparkContext.setLogLevel('WARN')
df = spark.read.text('../alice-in-wonderland.txt')
df.createOrReplaceTempView('book')
query = open('./sparkhelloworld/query.sql').read()

In [10]:
print(query)

with words as (
    select explode(split(lcase(regexp_replace(value, '[^a-zA-Z ]', '')), ' ')) as word 
    from book
)
select word, count(*) as ct
from words
where len(word) > 0
group by word
order by ct desc


In [17]:
from pyspark.sql.functions import *
words = df.select(explode(split(lower(regexp_replace("value", '[^a-zA-Z ]', '')), ' ')).alias('word'))
words.show()

+----------+
|      word|
+----------+
|    alices|
|adventures|
|        in|
|wonderland|
|          |
|        by|
|     lewis|
|   carroll|
|          |
|       the|
|millennium|
|   fulcrum|
|   edition|
|          |
|          |
|  contents|
|          |
|          |
|   chapter|
|         i|
+----------+
only showing top 20 rows



In [23]:
counts = words.where(length("word") > 0).groupBy("word").count().orderBy(desc("count"))
counts.show()

+-----+-----+
| word|count|
+-----+-----+
|  the| 1815|
|  and|  911|
|   to|  802|
|    a|  690|
|   of|  626|
|   it|  538|
|  she|  538|
| said|  462|
|  you|  430|
|   in|  427|
|    i|  401|
|alice|  385|
|  was|  359|
| that|  291|
|   as|  272|
|  her|  248|
| with|  226|
|   at|  220|
|   on|  204|
|  all|  197|
+-----+-----+
only showing top 20 rows

